In [1]:
import os

In [17]:
%pwd

'c:\\Users\\cisco\\Python_Projects\\ML+AI\\Pneumonia-Detection'

In [3]:
os.chdir("../")

In [42]:
%pwd

'c:\\Users\\cisco\\Python_Projects\\ML+AI\\Pneumonia-Detection'

In [43]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/jcarmfran/Pneumonia-Detection.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="jcarmfran"
os.environ["MLFLOW_TRACKING_PASSWORD"]="46bb34634c40407059f905e4d17d0d6c9c39abb1"

In [44]:
from dotenv import load_dotenv

load_dotenv() # load environment variables

os.environ["MLFLOW_TRACKING_URI"]=os.getenv("MLFLOW_TRACKING_URI")
os.environ["MLFLOW_TRACKING_USERNAME"]=os.getenv("MLFLOW_TRACKING_USERNAME")
os.environ["MLFLOW_TRACKING_PASSWORD"]=os.getenv("MLFLOW_TRACKING_PASSWORD")

## Loading the Model

In [45]:
import tensorflow as tf

In [46]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

## Entity Initialization

In [47]:
from dataclasses import dataclass
from pathlib import Path

In [48]:
@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

## Configuration

In [49]:
from pneuDetection.constants import *
from pneuDetection.utils.common import read_yaml, create_directories, save_json

In [50]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])


    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/xrays",
            mlflow_uri="https://dagshub.com/jcarmfran/Pneumonia-Detection.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

## Components

In [51]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [52]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config


    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod # no self; can be accessed 
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)


    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()


    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)


    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="ResNet50v2")
            else:
                mlflow.keras.log_model(self.model, "model")

## Pipeline

In [53]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

[2024-04-19 16:09:32,808: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-19 16:09:32,811: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-19 16:09:32,813: INFO: common: created directory at: artifacts]
Found 1568 images belonging to 2 classes.
25/25 [==============================] - 23s 864ms/step - loss: 7.0129 - accuracy: 0.7423
[2024-04-19 16:09:58,167: INFO: common: json file saved at: scores.json]
[2024-04-19 16:10:41,047: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'ProtocolError('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))': /jcarmfran/Pneumonia-Detection.mlflow/api/2.0/mlflow/runs/create]


2024/04/19 16:10:42 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-04-19 16:10:44,185: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 13). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\cisco\AppData\Local\Temp\tmp_w6zg5bh\model\data\model\assets
[2024-04-19 16:10:44,850: INFO: builder_impl: Assets written to: C:\Users\cisco\AppData\Local\Temp\tmp_w6zg5bh\model\data\model\assets]


Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/04/19 16:12:18 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 6
Created version '6' of model 'VGG16Model'.
